In [1]:
from pandas import Series, DataFrame
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
labels = pd.read_csv('labels.txt', sep = '\t', names = ['turk', 'URL', 'Category'])
gold = pd.read_csv('gold.txt', sep = "\t", names = ["URL", "Category"])

In [3]:
df=labels.merge(gold,how='left',left_on='URL',right_on='URL',suffixes=['_label','_gold'])

labels_on_gold=df[df['Category_gold'].notna()]
label_unknown=df[df['Category_gold'].isna()]

In [13]:
correct = labels_on_gold.copy()

correct['goodrate'] = (correct['Category_gold'] == correct['Category_label']).astype(int)
mask = (labels_on_gold['URL'] == gold.iloc[0]['URL'])*1

rater_goodness = correct.groupby('turk')['goodrate'].agg(['count','mean'])
rater_goodness2 = rater_goodness[rater_goodness['count']>=20]

You need to have the names as mean and count again to porperly run this regression, I dont know why the renamed variables did not work. 

In [14]:
import statsmodels.api as sm
from patsy import dmatrices

y, X = dmatrices('mean ~ count', data = rater_goodness, return_type = 'dataframe')

model = sm.OLS(y,X)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                   mean   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.5488
Date:                Tue, 03 Aug 2021   Prob (F-statistic):              0.459
Time:                        17:08:53   Log-Likelihood:                -94.663
No. Observations:                 269   AIC:                             193.3
Df Residuals:                     267   BIC:                             200.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.6431      0.024     27.071      0.0

rater_goodness2 is my variable for those that have more than 20 ratings. 

In [7]:
y, X = dmatrices('mean ~ count', data = rater_goodness2, return_type = 'dataframe')

model = sm.OLS(y,X)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                   mean   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.020
Method:                 Least Squares   F-statistic:                  0.004607
Date:                Tue, 03 Aug 2021   Prob (F-statistic):              0.946
Time:                        17:03:27   Log-Likelihood:                 29.366
No. Observations:                  52   AIC:                            -54.73
Df Residuals:                      50   BIC:                            -50.83
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.6848      0.034     20.129      0.0

- The regression is telling us there is no significant predictive power of the model when trying to predict the mean from count. The model with the turks that have rated more than 20 URLs is very slightly better but not meaningful in anyway. Probably just because there is less variation in this limited set. 